In [3]:
import pandas as pd
import numpy as np
import json
import ast


df1 = pd.read_csv('Kickstarter.csv')

photo_link = []
country = []
state = []
town = []
loc_type = []
project_url = []
reward_url =[]
list_category = []
list_subcategory = []

for i in range(len(df1)):
    photo_dict =df1.iloc[i].photo
    photo_link.append(ast.literal_eval(photo_dict)['full'])
    
for i in range(len(df1)):
    try:
        category, subcategory  = json.loads(df1.iloc[i].category)['slug'].split('/')
        list_category.append(category)
        list_subcategory.append(subcategory)
    except Exception as e:
        list_category.append(None)
        list_subcategory.append(None)

for i in range(len(df1)):
    try:
        loc_dict = json.loads(df1.iloc[i].location)
        country.append(str(loc_dict['country']))
    except Exception as e:
        country.append("None")
        
for i in range(len(df1)):
    try:
        loc_dict = json.loads(df1.iloc[i].location)
        state.append(str(loc_dict['state']))
    except Exception as e:
        state.append("None")
for i in range(len(df1)):
    try:
        loc_dict = json.loads(df1.iloc[i].location)
        town.append(str(loc_dict['name']))
    except Exception as e:
        town.append("None")
for i in range(len(df1)):
    try:
        loc_dict = json.loads(df1.iloc[i].location)
        loc_type.append(str(loc_dict['type']))
    except Exception as e:
        loc_type.append("None")

for i in range(len(df1)):
    try:
        project_url.append(str(json.loads(df1.iloc[i]['urls'])['web']['project']))
    except Exception as e:
        project_url.append("None")
for i in range(len(df1)):
    try:
        reward_url.append(str(json.loads(df1.iloc[i]['urls'])['web']['rewards']))
    except Exception as e:
        reward_url.append("None")        


df1['clean_category'] = np.array(list_category)
df1['subcategory'] = np.array(list_subcategory)
df1['photo_link'] = np.array(photo_link)
df1['clean_country'] = np.array(country)
df1['clean_state'] = np.array(state)
df1['town'] = np.array(town)
df1['location_type'] = np.array(loc_type)
df1['project_url'] = project_url
df1['reward_url'] = reward_url
df1.state.replace('1',1,inplace=True)
df1.state.replace('0',0,inplace=True)
df1.state.replace('live',0,inplace=True)
df1.state.replace('suspended',0,inplace=True)
df1.state.replace('canceled',0,inplace=True)
df1.rename(columns = {'state':'current_status'}, inplace = True)

df1.drop(['category','urls','photo','location','profile', 'creator',
'currency_symbol',
'currency_trailing_code',
'current_currency',
'disable_communication', 
'fx_rate',
'friends',
'is_backing',
'is_starred'],axis=1,inplace=True)


In [4]:
df1.to_csv('cleaned_data_kick.csv')

In [5]:
df1.head()

,backers_count,blurb,converted_pledged_amount,country,created_at,currency,deadline,goal,id,is_starrable,...,permissions,clean_category,subcategory,photo_link,clean_country,clean_state,town,location_type,project_url,reward_url
0,1,"Monsters, Fantasy, Illusion, Delusion, and a h...",20,US,1332493397,USD,1336447572,5400.0,2016865793,False,...,NaN,art,digital art,https://ksr-ugc.imgix.net/assets/011/334/766/6...,US,CO,Denver,Town,https://www.kickstarter.com/projects/987604337...,https://www.kickstarter.com/projects/987604337...
1,37,Nano Art will make and market customized piece...,1974,US,1332823105,USD,1337287105,5000.0,120596924,False,...,NaN,art,digital art,https://ksr-ugc.imgix.net/assets/011/336/473/c...,US,CA,Pasadena,Town,https://www.kickstarter.com/projects/300795038...,https://www.kickstarter.com/projects/300795038...
2,81,Video and audio coverage of the MUTEK festival...,4845,US,1331241234,USD,1337227140,20000.0,694989709,False,...,NaN,art,digital art,https://ksr-ugc.imgix.net/assets/011/326/978/4...,CA,QC,Montreal,Town,https://www.kickstarter.com/projects/29825942/...,https://www.kickstarter.com/projects/29825942/...
3,95,"Finally, A Storyboard App Done Right!",2948,US,1332350493,USD,1338448438,20000.0,1254591807,False,...,NaN,art,digital art,https://ksr-ugc.imgix.net/assets/011/333/805/3...,US,IL,Chicago,Town,https://www.kickstarter.com/projects/rotrier/s...,https://www.kickstarter.com/projects/rotrier/s...
4,10,"If you like books and bookmarks, stick these s...",522,US,1333768943,USD,1338609540,6000.0,1162595888,False,...,NaN,art,digital art,https://ksr-ugc.imgix.net/assets/011/342/195/d...,US,NJ,East Orange,Town,https://www.kickstarter.com/projects/201489045...,https://www.kickstarter.com/projects/201489045...


In [17]:
from bs4 import BeautifulSoup # documentation available at : #www.crummy.com/software/BeautifulSoup/bs4/doc/
from BeautifulSoup import NavigableString, Tag
import requests # To send http requests and access the page : docs.python-requests.org/en/latest/
import csv # To create the output csv file
import unicodedata # To work with the string encoding of the data
cleaned_df = pd.read_csv('cleaned_data001.csv')
projects_url = cleaned_df['project_url'].tolist()
list_description =[]
list_rewards = []
    
for url in projects_url:
    try:
        r = requests.get(url, timeout = 10) #Sending a request to access the page
    except Exception,e:
        break
    data = r.text
    soup = BeautifulSoup(data) # Getting the page source into the soup
    ps = []
    for div in soup.find_all('div'):
        if (div.get('class') != None and div.get('class')[0] == 'full-description'):
            ps = div.find_all('p')
        if (div.get('class') != None and div.get('class')[0] == 'NS_projects__rewards_list'):
            li_len = len(div.find_all('li')) -1
        desc = ' ' 
        for i in ps:
            desc+= i.text
    list_rewards.append(li_len)
    list_description.append(desc.encode('utf-8'))


KeyboardInterrupt: 

In [ ]:
cleaned_df['Description'] = np.array(list_description)
cleaned_df['Rewards'] = np.array(list_rewards)

In [18]:
len(list_description
   )

2340

In [19]:
len(list_rewards)

2340

In [21]:
the_file = open('descriptions.txt', 'w')
for item in list_description:
    the_file.write("%s\n" % item)

In [23]:
cleaned_panel = df1[0:2340]

In [24]:
cleaned_panel['Description'] = np.array(list_description)

/Users/saurabh/venv/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [25]:
cleaned_panel['Rewards'] = np.array(list_rewards)

/Users/saurabh/venv/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [26]:
cleaned_panel.to_csv('Cleaned_data_with_desc_and_rewards.csv')